In [ ]:
import numpy as np
import pycycle as cy
import matplotlib.pyplot as plt
import scipy
%load_ext autoreload
%autoreload 2
%matplotlib notebook

Mesh creation
===========

In [ ]:
b1 = (0, 40.0)
b2 = (0, 50.0)
h1 = 0.2
h2 = 1.0
star_centre = (1, 1)
normal1 = cy.mesh.line_normal((0, 0), b1, star_centre)
normal2 = cy.mesh.line_normal(b1, b2, star_centre)

# Create a mesh from origin to b1 with resolution h1
# True means on-fault
mesh = cy.mesh.tessellate_line((0, 0), b1, h1, normal1, True)
# Add a mesh from b1 to b2 with resolution h2
mesh = mesh + cy.mesh.tessellate_line(b1, b2, h2, normal2)
# Extend to infinity
mesh.append(cy.mesh.InfiniteLineElement(b2, normal2))

Parameters
==========

In [ ]:
rho = 2.670   # density [g/m^3]
v_s = 3.464   # shear wave velocity [km/s]
Vp = 1e-9     # plate rate [m/s]
V0 = 1e-6     # reference slip rate [m/s]
b = 0.015     # b parameter
L = 0.008     # critical slip distance [m]
f0 = 0.6      # reference friction coefficient
sn = 50       # normal stress [MPa]
Vinit = 1e-9  # initial slip rate [m/s]
cp = cy.seas.ConstantParams(rho, v_s, Vp, V0, b, L, f0, sn, Vinit)

In [ ]:
a0 = 0.010
amax = 0.025
# a parameter depends on x
def a(x):
    z = x[1]
    H = 15.0
    h = 3.0
    if z < H:
        return a0
    elif z < H + h:
        return a0 + (amax - a0) * (z - H) / h
    else:
        return amax

# pre-stress may depend on position, constant here
def tau_pre(x):
    e = np.exp((f0 + b * np.log(V0 / Vinit)) / amax)
    return -(sn * amax * np.arcsinh((Vinit / (2.0 * V0)) * e) + cp.eta * Vinit)

vp = cy.seas.VariableParams(mesh, a, tau_pre)

Compile
==========

In [ ]:
# initialize solver context
ctx = cy.seas.Context(mesh, cy.green.G_fs, cy.green.dG_fs_dn, vp, cp)

Plot setup
========

In [ ]:
u_fig = plt.figure()

# fig.canvas.set_window_title('Canvas active title')
u_fig.suptitle('Cumulative slip (m)', fontsize=20)

# Create plots inside the figures
u_ax = u_fig.add_subplot(111)

In [ ]:
v_fig = plt.figure()

# fig.canvas.set_window_title('Canvas active title')
v_fig.suptitle('Log10(v) (m/s) vs. time (s)', fontsize=20)

# Create plots inside the figures
v_ax = v_fig.add_subplot(111)

In [ ]:
thresholds = [
    {'color': '#000000', 'vthresh': 0,     'dt': 10*365*24*60*60, 'interval': '10 years' },
    {'color': '#ff0000', 'vthresh': 1e-06, 'dt': 24*60*60,        'interval': '1  day'   },
    {'color': '#ffcc00', 'vthresh': 1e-02, 'dt': 1,               'interval': '1  second'}
]

Initial conditions
=============

In [ ]:
y0 = cy.seas.y0(ctx) # initial condition

In [ ]:
t0 = 0
tend = 8e9

Solve
=====

In [ ]:
monitor = cy.monitor.Monitor(thresholds, u_ax, u_fig, v_ax, v_fig)

def F(t, y, ctx):
    # a new step begins, take y to represent the result of last step.
    fresh = t == F.t_
    F.t_  = t
    return cy.seas.F(t,y,ctx,monitor) if fresh else cy.seas.F(t,y,ctx)
F.t_ = t0

result = scipy.integrate.solve_ivp( F, (t0, tend), y0, method='RK45', rtol=1e-7, atol=1e-7, args=(ctx,), first_step=100) #, max_step=60*60*24*365

Postprocessing
=========